Lets recall the structure of our modified train data 

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np

m_train = pd.read_csv("mod_train.csv", encoding = "ISO-8859-1", index_col=0)

print(m_train.dtypes,"\n")

print(m_train.shape,"\n")

print(m_train.label.value_counts(),"\n")

id                            int64
additionalAttributes         object
label                         int64
format                       object
studio                       object
language                     object
actors                       object
movie_genre                  object
performer                    object
upc                          object
store_item_number_(dpci)     object
tcin                        float64
release_date                 object
publisher                    object
genre                        object
number_of_pages              object
size                         object
color                        object
asin                         object
material                     object
dtype: object 

(300000, 20) 

2    225376
0     27924
3     23376
1     23324
Name: label, dtype: int64 



C:\Users\balaji\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Lets see the accurancy of Mod_train csv and its confustion matrix

In [ ]:
columns = list(m_train.columns)
columns.remove('id')
columns.remove('label')
columns.remove("additionalAttributes")

for x in columns:
    m_train[x] = (m_train[x].notnull()).astype('int')

#print(train.columns.ndarray)
X = m_train.as_matrix(columns)
y = m_train.label

# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

# training a linear SVM classifier
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)

# model accuracy for X_test 
accuracy = svm_model_linear.score(X_test, y_test)
print (accuracy)

cm = confusion_matrix(y_test, svm_predictions)

print (cm)

As the index mapping for the label is like below

0 -> books, 1 -> music, 2 -> rest, 3 -> videos

      0        1    2      3 

0  [[ 5206     0  1731    33]
1   [    1  4201  1610    61]
2   [  114   222 56108     4]
3   [   10    76  1290  4333]]

From the confusion matrix we can infer that the predictions of books, videos , musics are proper but some of those are wrongly classified as rest.

So we need to add some for usefull features that are capable of classifiying the features even more

As per our previous approach we will first identify the top features in each category of label and identify which are those are usefull

In [10]:
from collections import Counter
train = pd.read_csv("bmv_training_set.csv")
set_label = set(train["label"].values)
for choices in set_label:
    k = list()
    print("\n",choices,"\n")
    text = train["additionalAttributes"][train["label"] == choices].values
    res = ";".join(text)
    text = res.split(";")
    text = [i.split('=')[0].replace(":","").replace(" ","_").lower() for i in text] 
    for x , v in Counter(text).most_common(100):
       k.append(x)
    print(k)


 videos 

['format', 'studio', 'language', 'actors', 'movie_genre', 'upc', 'directors', 'rating', 'store_item_number_(dpci)', 'tcin', 'asin', 'dimensions', 'weight', 'author', 'genre', 'average_customer_review', 'origin', 'software_format', 'amazon_best_sellers_rank', 'region', 'country_of_origin', 'movie_studio', 'release_date', 'rated', 'run_time', 'duration', 'dvd_release_date', 'number_of_discs', 'edition', 'aspect_ratio', 'movie_category', 'original_languages', 'stars', 'mpaa_rating', 'subtitles', 'movie_mpaa_rating', 'brand', 'record_label', 'color', 'movie_picture_format', 'subtitle_language', 'condition', 'producers', 'writers', 'screen_format', 'director', 'manufacturer_part_number', 'dubbed', 'street_date', 'container_type', 'assembled_product_weight', 'extra_info', 'features', 'studio_name', 'audio_tracks', 'run_time_(in_minutes)', 'digital_video_formats', 'starring', 'number_of_tapes', 'model', 'vhs_release_date', 'voices', 'tv_rating', 'series_title', 'character', 'target

From the above chunck of data we can infer that 

1. "Music" label has some features which are numeric 1.,2.,3. extra
2. "Rest" label has some features which are redendent like "type0", "type1", "type2", "feature_1", "feature_2", .. 


### Identifying Hyper-parameters

Lets again dig into further more like what are the common words in the set of features of each label

In [12]:
for choices in set_label:
    k = list()
    print("\n",choices,"\n")
    text = train["additionalAttributes"][train["label"] == choices].values
    res = ";".join(text)
    text = res.split(";")
    text = [i.split('=')[0].replace(":","").replace(" ","_").lower() for i in text] 
    for x , v in Counter(text).most_common(100):
       k.append(x)  
    m = [i.split('_')[0] for i in k]
    print(Counter(m).most_common(10))


 videos 

[('', 8), ('movie', 6), ('audio', 3), ('studio', 2), ('run', 2), ('number', 2), ('model', 2), ('format', 1), ('language', 1), ('actors', 1)]

 music 

[('pop', 3), ('classical', 2), ('upc', 1), ('release', 1), ('label', 1), ('record', 1), ('performer', 1), ('store', 1), ('tcin', 1), ('author', 1)]

 rest 

[('feature', 12), ('warranty', 4), ('color', 2), ('shipping', 2), ('model', 2), ('care', 2), ('brand', 2), ('manufacturer', 2), ('item', 2), ('product', 2)]

 books 

[('series', 4), ('edition', 4), ('number', 3), ('book', 3), ('publisher', 2), ('format', 2), ('genre', 2), ('amazon', 2), ('age', 2), ('lexile', 2)]


So in the list of features in each category there are some features which has a starting word common. So out of 100 top features in each category which has more than 1 occurences in the feature names

In [13]:
for choices in set_label:
    k = list()
    new_m= list()
    print("\n",choices,"\n")
    text = train["additionalAttributes"][train["label"] == choices].values
    res = ";".join(text)
    text = res.split(";")
    text = [i.split('=')[0].replace(":","").replace(" ","_").lower() for i in text] 
    for x , v in Counter(text).most_common(100):
       k.append(x)  
    m = [i.split('_')[0] for i in k]
    for a, b in Counter(m).most_common(10):
        if b > 1 and a != "":
            new_m.append(a)
    print(new_m)


 videos 

['movie', 'audio', 'studio', 'run', 'number', 'model']

 music 

['pop', 'classical']

 rest 

['feature', 'warranty', 'color', 'shipping', 'model', 'care', 'brand', 'manufacturer', 'item', 'product']

 books 

['series', 'edition', 'number', 'book', 'publisher', 'format', 'genre', 'amazon', 'age', 'lexile']


So we will combine the feature names that are starting with movie into a single feature "movie". For example if a videos category additionalAttributes has the following value

movie genre="abcd";movie name="abcd";

We will set the movie column for that row to be 1 to denote that the value contains a strong feature "movie"

We will combine all those value into a single list

In [21]:
extract = list()
features = list()
for choices in set_label:
    k = list()
    text = train["additionalAttributes"][train["label"] == choices].values
    res = ";".join(text)
    text = res.split(";")
    text = [i.split('=')[0].replace(":","").replace(" ","_").replace(".","").lower() for i in text] 
    for x , v in Counter(text).most_common(100):
       k.append(x)  
    features.extend(k)
    m = [i.split('_')[0] for i in k]
    for a, b in Counter(m).most_common(10):
        if b > 1 and a != "":
            extract.append(a)
print(extract)

['movie', 'audio', 'studio', 'run', 'number', 'model', 'pop', 'classical', 'feature', 'warranty', 'color', 'shipping', 'model', 'care', 'brand', 'manufacturer', 'item', 'product', 'series', 'edition', 'number', 'book', 'publisher', 'format', 'genre', 'amazon', 'age', 'lexile']


Next step to identify the common features that appears in all the category. Though the feature appears to have more count if it presents in all the four category it won't be a valued feature

In [24]:
poor_features = list()
for u , v in Counter(features).most_common(50):
    if (v == 4):
        poor_features.append(u)
print(poor_features)


['upc', 'store_item_number_(dpci)', 'tcin', 'asin', 'average_customer_review', 'origin', 'amazon_best_sellers_rank', 'brand', 'manufacturer_part_number', 'assembled_product_weight', 'features']


So we got the "poor_features", overall features in "features" and substring feature that appeared in top 10 as "extract". Next steps would be

1. Removing duplicates from "features"
2. Removing the feautres present in "poor_features" list
3. Comibing the "extract" to the feautres and forming the "final_features"

In [28]:
#Once again getting the features which has duplicates in it
import re
d_features = list() # Collection of features
for choices in set_label:
    k = list()
    text = train["additionalAttributes"][train["label"] == choices].values
    res = ";".join(text)
    text = res.split(";")
    text = [i.split('=')[0].replace(":","").replace(" ","_").replace(".","").lower() for i in text] 
    for x , v in Counter(text).most_common(20):
        x = re.sub('\d+', '', x) #removing digits from features 
        x = x.strip(":-,")  #removing special characters
        x = re.sub('_.*', '', x) #removing characters after _ to avoid duplicates from "extract"
        k.append(x)  
    d_features.extend(k)

d_features.extend(extract) # Adding extracted data to the list of features

n_features= list()
feature_hash = Counter(d_features)
for u , v in feature_hash.items():
    n_features.append(u) # Removing Duplicates 

final_features = [x for x in n_features if x not in poor_features]
print(len(final_features))#Final list of features after removing the poor_features 
print(final_features)

56
['format', 'studio', 'language', 'actors', 'movie', 'directors', 'rating', 'store', 'dimensions', 'weight', 'author', 'genre', 'average', 'software', 'amazon', 'region', 'release', 'label', 'record', 'performer', 'catalognumber', '', 'pop', 'artist', 'number', 'rank', 'color', 'size', 'material', 'country', 'shipping', 'model', 'care', 'manufacturer', 'item', 'sizing', 'type', 'publisher', 'isbn', 'pages', 'product', 'publication', 'book', 'audience', 'height', 'wdth', 'thickness', 'audio', 'run', 'classical', 'feature', 'warranty', 'series', 'edition', 'age', 'lexile']


Now lets add these features into our train data set. 

### Trying different approach to reduce the time taken using CountVectorizer

In [32]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
train = pd.read_csv("bmv_training_set.csv", index_col=0)
print(train.shape)
final_features.appendadd_attr = train["additionalAttributes"].values

#new = list()
text_n = list()
for text in add_attr:
    text = text.split(";")
    text = [i.split('=')[0].replace(":","").replace(" ","_").replace(".","").lower().strip("?:;.'") for i in text]
    text = [re.sub('_.*', '', i) for i in text]
    text = [re.sub('^\d+','track',i) for i in text]
    text = [re.sub('\d','',i) for i in text]
    text = list(set(text))
    text = " ".join(text)
    text_n.append(text)

print(text_n[16])
dtm = vect.transform(text_n)
print(dtm.toarray()[16])
new = pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())
train.update(new)
print(train.head(20))

#train.to_csv("modio_train.csv")('track') #To denote the Tracks in music category
vect.fit(final_features)
features = vect.get_feature_names()
train = train.rename(columns={'label': 'final_label'}) #Changing the target variable name as there is one feature "label" present
train = train.reindex(columns = np.append( train.columns.values, features))
print(train.head(20))

(300000, 2)
                                 additionalAttributes final_label  actors  \
id                                                                          
0   Store Item Number (DPCI)=244-02-6685;Pop Music...       music     NaN   
1   Package Dimensions=13.8 x 13 x 3.2 inches;Best...        rest     NaN   
2               color=GREY;size=medium;Sizing:=Junior        rest     NaN   
3   Product Dimensions=12 x 0 x 6 inches;ASIN=B00C...        rest     NaN   
4                                              size=7        rest     NaN   
5   Shipping Weight=5.8 ounces;ASIN=B00OVQRSCM;Ave...        rest     NaN   
6   style=Necklace - 18 inches;Origin=USA;Product ...        rest     NaN   
7   Compatibility=Car Radio;Feature_1=Material: AB...        rest     NaN   
8   Gemstone Shape=Oval and Round;Gemstone Cut=Ova...        rest     NaN   
9   Color Family=Brown;Features=strengthens nails,...        rest     NaN   
10           Origin of Components=USA and/or Imported        res

In [34]:
add_attr = train["additionalAttributes"].values

text_n = list()
for text in add_attr:
    text = text.split(";")
    text = [i.split('=')[0].replace(":","").replace(" ","_").replace(".","").lower().strip("?:;.'") for i in text]
    text = [re.sub('_.*', '', i) for i in text]
    text = [re.sub('^\d+','track',i) for i in text]
    text = [re.sub('\d','',i) for i in text]
    text = list(set(text))
    text = " ".join(text)
    text_n.append(text)

dtm = vect.transform(text_n)
new = pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())
train.update(new)
print(train.head())

train.to_csv("modified_train.csv")

                                 additionalAttributes final_label  actors  \
id                                                                          
0   Store Item Number (DPCI)=244-02-6685;Pop Music...       music     0.0   
1   Package Dimensions=13.8 x 13 x 3.2 inches;Best...        rest     0.0   
2               color=GREY;size=medium;Sizing:=Junior        rest     0.0   
3   Product Dimensions=12 x 0 x 6 inches;ASIN=B00C...        rest     0.0   
4                                              size=7        rest     0.0   

    age  amazon  artist  audience  audio  author  average   ...    sizing  \
id                                                          ...             
0   0.0     0.0     0.0       0.0    0.0     0.0      0.0   ...       0.0   
1   0.0     0.0     0.0       0.0    0.0     0.0      0.0   ...       0.0   
2   0.0     0.0     0.0       0.0    0.0     0.0      0.0   ...       1.0   
3   0.0     0.0     0.0       0.0    0.0     0.0      0.0   ...       0.0  

The above code took less than 3 minutes

### Training the Model

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
train = pd.read_csv("bmv_training_set.csv")
m_train = pd.read_csv("modified_train.csv", encoding = "ISO-8859-1")
test = pd.read_csv("modified_test.csv" , encoding = "ISO-8859-1", index_col=0)
columns = list(m_train.columns)
columns = columns[3:]

m_train["final_label"] = m_train["final_label"].astype('category').cat.codes #converting the categories into int
test["final_label"] = np.nan

slc = np.r_[columns]
m_train[slc] = m_train[slc].astype(int)# converting all the dtypes to in

print(m_train.head())

print(m_train.dtypes)

   id                               additionalAttributes  final_label  actors  \
0   0  Store Item Number (DPCI)=244-02-6685;Pop Music...            1       0   
1   1  Package Dimensions=13.8 x 13 x 3.2 inches;Best...            2       0   
2   2              color=GREY;size=medium;Sizing:=Junior            2       0   
3   3  Product Dimensions=12 x 0 x 6 inches;ASIN=B00C...            2       0   
4   4                                             size=7            2       0   

   age  amazon  artist  audience  audio  author   ...    sizing  software  \
0    0       0       0         0      0       0   ...         0         1   
1    0       0       0         0      0       0   ...         0         0   
2    0       0       0         0      0       0   ...         1         0   
3    0       0       0         0      0       0   ...         0         0   
4    0       0       0         0      0       0   ...         0         0   

   store  studio  thickness  track  type  warranty

### Navie Bayes

In [39]:
from sklearn.naive_bayes import GaussianNB
#print(train.columns.ndarray)
X = m_train.as_matrix(columns)
y = m_train.final_label

# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

gnb = GaussianNB().fit(X_train, y_train)
gnb_predictions = gnb.predict(X_test)
 
# accuracy on X_test
accuracy = gnb.score(X_test, y_test)
print (accuracy)

# creating a confusion matrix
cm = confusion_matrix(y_test, gnb_predictions)
print(cm)

0.832066666667
[[ 5783  1171    13     3]
 [   41  5809     1    22]
 [ 1912  5091 46453  2992]
 [   72  1261    16  4360]]


Seems like the accuracy improved. From the confusion matrix we can infer that the music category is wrongly classified 

0 -> books, 1 -> music, 2 -> rest, 3 -> videos

### Support Vector Machine

In [40]:
from sklearn.svm import SVC

#print(m_train.columns.ndarray)
X = m_train.as_matrix(columns)
y = m_train.final_label

# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

# training a linear SVM classifier
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)

# model accuracy for X_test 
accuracy = svm_model_linear.score(X_test, y_test)
print (accuracy)

# creating a confusion matrix
cm = confusion_matrix(y_test, svm_predictions)
print(cm)

0.9656
[[ 5745  1168    56     1]
 [   26  5769    38    40]
 [   14     1 56428     5]
 [   11  1108   112  4478]]


Now there is a good improvement in classifying the final_label. Need to fine tune the "music" label further

### Prepare submittion file

In [ ]:
test = pd.read_csv("bmv_test_set.csv")
vect.fit(features)
features = vect.get_feature_names()
test = test.rename(columns={'label': 'final_label'})
test = test.reindex(columns = np.append( test.columns.values, features))
print(test.head())

add_attr = test["additionalAttributes"].values

text_n = list()
for text in add_attr:
    text = text.split(";")
    text = [i.split('=')[0].replace(":","").replace(" ","_").replace(".","").lower().strip("?:;.'") for i in text]
    text = [re.sub('_.*', '', i) for i in text]
    text = [re.sub('^\d+','track',i) for i in text]
    text = [re.sub('\d','',i) for i in text]
    text = list(set(text))
    text = " ".join(text)
    text_n.append(text)

dtm = vect.transform(text_n)
new = pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())
test.update(new)
print(test.head())

test.to_csv("modified_test.csv")


m_train = pd.read_csv("modified_train.csv", encoding = "ISO-8859-1")
m_test = pd.read_csv("modified_test.csv" , encoding = "ISO-8859-1", index_col=0)
columns = list(m_train.columns)
columns = columns[3:]

m_train["final_label"] = m_train["final_label"].astype('category').cat.codes #converting the categories into int
m_test["final_label"] = np.nan

slc = np.r_[columns]
m_train[slc] = m_train[slc].astype(int)# converting all the dtypes to in
m_test[slc] = m_test[slc].astype(int)
       
       
from sklearn.svm import SVC
#print(train.columns.ndarray)
X_train = m_train.as_matrix(columns)
X_test = m_test.as_matrix(columns)

y_train = m_train.final_label
y_test = m_test.final_label

# dividing X, y into train and test data
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

# training a linear SVM classifier
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)

m_test["final_label"] = svm_predictions
m_test.to_csv("submission.csv")

In [6]:
sub = pd.read_csv("submission.csv", encoding = "ISO-8859-1", index_col=0)
columns = list(sub.columns)
columns = columns[2:58]
sub.drop(columns, axis=1, inplace=True)
sub.rename(columns={'final_label': 'label'}, inplace=True)
print(sub.head())
print(sub.shape)
sub.to_csv("final_submission.csv", index=False)

   id                               additionalAttributes  label
0   0  Movie Genre=Sports & Recreation;Store Item Num...      3
1   1  author=Francis Ford Coppola featuring Marlon B...      0
2   2  Performer=Phil Hamilton;Record Label=SELECT-O-...      1
3   7                                       Color=Chrome      2
4   9                          Format=Hardcover - $12.82      0
(60000, 3)
